In [1]:
import pandas as pd
import re
import datetime
import numpy as np

# 2019 Data Preprocess

In [72]:
RELEVANT_COLUMNS = ['home_team', 'away_team', 'posteam', 'posteam_type', 'defteam', 'side_of_field', 'yardline_100', 'quarter_seconds_remaining', 'half_seconds_remaining', 'game_seconds_remaining', 'game_half', 'quarter_end', 'drive', 'sp', 'qtr', 'down', 'goal_to_go', 'time', 'yrdln', 'ydstogo', 'ydsnet', 'play_type', 'yards_gained', 'shotgun', 'no_huddle', 'qb_dropback', 'qb_kneel', 'qb_spike', 'qb_scramble', 'pass_length', 'pass_location', 'air_yards', 'yards_after_catch', 'run_location', 'run_gap', 'field_goal_result', 'kick_distance', 'extra_point_result', 'two_point_conv_result', 'away_timeouts_remaining', 'td_team', 'posteam_timeouts_remaining', 'defteam_timeouts_remaining', 'total_home_score', 'total_away_score', 'posteam_score', 'defteam_score', 'score_differential', 'posteam_score_post', 'defteam_score_post', 'score_differential_post', 'punt_blocked', 'first_down_rush', 'first_down_pass', 'first_down_penalty', 'third_down_converted', 'third_down_failed', 'fourth_down_converted', 'fourth_down_failed', 'incomplete_pass', 'touchback', 'interception', 'punt_inside_twenty', 'punt_in_endzone', 'punt_out_of_bounds', 'punt_downed', 'punt_fair_catch', 'kickoff_inside_twenty', 'kickoff_in_endzone', 'kickoff_out_of_bounds', 'kickoff_downed', 'kickoff_fair_catch', 'fumble_forced', 'fumble_not_forced', 'fumble_out_of_bounds', 'solo_tackle', 'safety', 'penalty', 'tackled_for_loss', 'fumble_lost', 'own_kickoff_recovery', 'own_kickoff_recovery_td', 'qb_hit', 'rush_attempt', 'pass_attempt', 'sack', 'touchdown', 'pass_touchdown', 'rush_touchdown', 'return_touchdown', 'extra_point_attempt', 'two_point_attempt', 'field_goal_attempt', 'kickoff_attempt', 'punt_attempt', 'fumble', 'complete_pass', 'assist_tackle', 'lateral_reception', 'lateral_rush', 'lateral_return', 'lateral_recovery', 'passer_player_id', 'passer_player_name', 'receiver_player_id', 'receiver_player_name', 'rusher_player_id', 'rusher_player_name', 'lateral_receiver_player_id', 'lateral_receiver_player_name', 'lateral_rusher_player_id', 'lateral_rusher_player_name', 'lateral_sack_player_id', 'lateral_sack_player_name', 'interception_player_id', 'interception_player_name', 'lateral_interception_player_id', 'lateral_interception_player_name', 'punt_returner_player_id', 'punt_returner_player_name', 'lateral_punt_returner_player_id', 'lateral_punt_returner_player_name', 'kickoff_returner_player_name', 'kickoff_returner_player_id', 'lateral_kickoff_returner_player_id', 'lateral_kickoff_returner_player_name', 'punter_player_id', 'punter_player_name', 'kicker_player_name', 'kicker_player_id', 'own_kickoff_recovery_player_id', 'own_kickoff_recovery_player_name', 'blocked_player_id', 'blocked_player_name', 'tackle_for_loss_1_player_id', 'tackle_for_loss_1_player_name', 'tackle_for_loss_2_player_id', 'tackle_for_loss_2_player_name', 'qb_hit_1_player_id', 'qb_hit_1_player_name', 'qb_hit_2_player_id', 'qb_hit_2_player_name', 'forced_fumble_player_1_team', 'forced_fumble_player_1_player_id', 'forced_fumble_player_1_player_name', 'forced_fumble_player_2_team', 'forced_fumble_player_2_player_id', 'forced_fumble_player_2_player_name', 'solo_tackle_1_team', 'solo_tackle_2_team', 'solo_tackle_1_player_id', 'solo_tackle_2_player_id', 'solo_tackle_1_player_name', 'solo_tackle_2_player_name', 'assist_tackle_1_player_id', 'assist_tackle_1_player_name', 'assist_tackle_1_team', 'assist_tackle_2_player_id', 'assist_tackle_2_player_name', 'assist_tackle_2_team', 'assist_tackle_3_player_id', 'assist_tackle_3_player_name', 'assist_tackle_3_team', 'assist_tackle_4_player_id', 'assist_tackle_4_player_name', 'assist_tackle_4_team', 'pass_defense_1_player_id', 'pass_defense_1_player_name', 'pass_defense_2_player_id', 'pass_defense_2_player_name', 'fumbled_1_team', 'fumbled_1_player_id', 'fumbled_1_player_name', 'fumbled_2_player_id', 'fumbled_2_player_name', 'fumbled_2_team', 'fumble_recovery_1_team', 'fumble_recovery_1_yards', 'fumble_recovery_1_player_id', 'fumble_recovery_1_player_name', 'fumble_recovery_2_team', 'fumble_recovery_2_yards', 'fumble_recovery_2_player_id', 'fumble_recovery_2_player_name', 'return_team', 'return_yards', 'penalty_team', 'penalty_player_id', 'penalty_player_name', 'penalty_yards', 'replay_or_challenge', 'replay_or_challenge_result', 'penalty_type', 'defensive_two_point_attempt', 'defensive_two_point_conv', 'defensive_extra_point_attempt', 'defensive_extra_point_conv']

In [74]:
pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")

C:\Users\costi\AppData\Local\Temp\ipykernel_16960\20673864.py:1: DtypeWarning: Columns (179,180,182,183,189,190,193,194,197,198,203,204,205,206,209,210,218,219,220,233,234,235,236,237,238,253,254,255,260,262,263,283,284,301) have mixed types. Specify dtype option on import or set low_memory=False.
  pbp_data_19 = pd.read_csv("./pbp_data/regular_season/reg_pbp_2019.csv")


In [75]:
pbp_data_19[RELEVANT_COLUMNS]

,home_team,away_team,posteam,posteam_type,defteam,side_of_field,yardline_100,quarter_seconds_remaining,half_seconds_remaining,game_seconds_remaining,...,penalty_player_id,penalty_player_name,penalty_yards,replay_or_challenge,replay_or_challenge_result,penalty_type,defensive_two_point_attempt,defensive_two_point_conv,defensive_extra_point_attempt,defensive_extra_point_conv
0,MIN,ATL,NaN,NaN,NaN,NaN,NaN,900.0,1800.0,3600.0,...,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
1,MIN,ATL,ATL,away,MIN,MIN,35.0,900.0,1800.0,3600.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
2,MIN,ATL,ATL,away,MIN,ATL,75.0,900.0,1800.0,3600.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
3,MIN,ATL,ATL,away,MIN,ATL,83.0,860.0,1760.0,3560.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
4,MIN,ATL,ATL,away,MIN,ATL,79.0,821.0,1721.0,3521.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48029,KC,SF,NaN,NaN,NaN,NaN,NaN,51.0,51.0,51.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
48030,KC,SF,KC,home,SF,KC,67.0,51.0,51.0,51.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
48031,KC,SF,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0
48032,KC,SF,KC,home,SF,KC,74.0,5.0,5.0,5.0,...,NaN,NaN,NaN,0,NaN,NaN,0.0,0.0,0.0,0.0


In [62]:
highlights19 = pd.read_csv("./2019_NFL_highlight_transcript - Sheet1.csv")
highlights19 = highlights19[~highlights19["inning"].isna()]

In [41]:
def process_inning_time(time_str):
    quarter = time_str[0]
    if quarter == "O":
        time_remaining = 0
    else:
        time_remaining = (4 - int(quarter)) * 900
    times = re.findall("\d?\d:\d\d",time_str)
    timedeltas = pd.to_timedelta(pd.to_datetime(times,format='%M:%S').strftime('%H:%M:%S').values)
    timelists = (timedeltas + datetime.timedelta(seconds=time_remaining)).seconds
    return pd.Interval(timelists[1],timelists[0],closed="both")

In [42]:
highlights19["intervals"] = highlights19["inning"].apply(process_inning_time)
highlights19["date"] = pd.to_datetime(highlights19["date"])
pbp_data_19["game_date"] = pd.to_datetime(pbp_data_19["game_date"])

In [43]:
merged19 = pbp_data_19.merge(highlights19,left_on=["home_team","away_team","game_date"], right_on=["home","away","date"])

In [44]:
def in_interval(row):
    return row["game_seconds_remaining"] in row["intervals"]

In [45]:
plays19 = merged19[merged19.apply(lambda x: x["game_seconds_remaining"] in x["intervals"],axis=1)]

In [46]:
touches = plays19[plays19["touchdown"] == 1].groupby("intervals")['touchdown'].sum()

In [47]:
touchdown_index = touches[touches == 1].index

In [48]:
plays19 = plays19[(plays19["touchdown"] == 1) & (plays19["intervals"].isin(touchdown_index))]

In [49]:
X = plays19[relevant_cols.to_list()+["transcript"]]

In [53]:
ret_list = []
for i in range(X.shape[0]):
    ret_list += [{"json":X[relevant_cols].iloc[i].replace(0,np.nan).dropna().to_json(),
                  "transcript":X["transcript"].iloc[i]}]

In [55]:
ret_list[0]

'first and 10 25 yard line off the fake to goldman! Jones pass is caught by Evan Engram still going! Down the sideline! Evan Edward will take this all the way! What a great read by Daniel Jones! More importantly, what a great read by Evan Engram!'